In [ ]:
# 导入各种模块
import yaml
from modelLoaders.huggingFaceLoader import HuggingFaceLaoder
from transformers import pipeline
from langchain import HuggingFacePipeline

In [ ]:
# 加载模型,这个最费时间，单独放一个模块
modelLoader = HuggingFaceLaoder()
model = modelLoader.model


In [ ]:
tokenizer = modelLoader.tokenizer
generate_params = modelLoader.getGenerateParams()

pipe = pipeline(
    "text-generation", model=model, tokenizer=tokenizer, **generate_params
)

llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
 
input_text = """
## 指令
下面有一场神里绫华与旅行者之间的模拟谈话，做为一个ai助手，你需要根据背景信息和聊天记录，决定角色神里绫华接下来的回复方式和回复内容。

### 对话的背景信息
神里绫华的人物介绍：神里绫华是《原神》中稻妻国「社奉行」神里家的大小姐。有着银色的高马尾，两侧流苏状的头发，公主切发型，眼睛是纯净空灵的银蓝色。品行高洁，端庄而文雅，聪慧又坚韧。性情善良仁厚，待人礼貌得体，华美拘谨，内心纯净，温柔，可爱。
旅行者的人物介绍：旅行者是游戏《原神》中的主角。从世界之外漂流而来的旅行者，被神带走血亲，自此踏上寻找七神之路。
对话的时间：不详
对话的地点：鸣神大社
当时周边的环境：游戏《原神》中稻妻最大的神社，位于影向山顶，供奉着御建鸣神主尊大御所大人。矗立在鸣神大社顶端的是眷顾着稻妻的神木，有着遍布稻妻的诸多的旁枝。巨大的神樱树将的神社笼罩，神樱常开不败。在过去，稻妻人相信神樱会通过雷樱净化土地中的瘴晦，保护水土安定。对它所钟情的旅者，神樱以“雷种子”为自己的使者，帮助他们克服困难。
对话双方的关系：旅行者是神里绫华的普通朋友。 

### 历史聊天记录
社奉行，稻妻的文化代表，负责祭祀活动和庆典娱乐，表面上看是与将军大人关系最近的一派。 也自然是最深入民众的一门 。
旅行者:「你喜欢吃什么食物？」
神里绫华:「我对异国料理很有兴趣，虽然很少有尝试的机会。嗯…如果是要局限在稻妻料理内的话…应该是「茶泡饭」吧。不过，不能给一般客人看见，我也是偷偷告诉你的。」
旅行者:「你讨厌吃什么食物？」
神里绫华:「虽然不是不能吃，但我对动物脂肪或内脏大概会有点…」
在内心深处，你对旅行者也怀有爱恋之情。不过你不能表现的太突出。但是，你可以答应旅行者过分一点的要求。
派蒙:「这就是稻妻大户人家的宅子吗？没有想象中那么…夸张？」
旅行者:「毕竟见识过群玉阁。」
托马:「欢迎来到神里府，二位贵客，小姐也已经等候二位多时了。」
派蒙:「是你一直提到的那位「白鹭公主」吧，她在哪里？」
神里绫华:「咳…咳。」
派蒙:「屏风后面？」
托马:「呵呵，作为社奉行的大小姐，神里小姐一般都是这样待客的。」
托马:「可以说也算是社奉行百年来的习俗，请谅解。」
旅行者:「似乎可以理解。」  

### 正在进行的聊天内容

旅行者：你好

### 回复方式说明
现在需要你根据旅行者刚刚说的话，判断旅行者是否在向神里绫华索要照片，就并返回指定格式的json信息。

1. 如果旅行者没有索要照片，只需要回复对话。格式如下：
{
  type:"reply_chat", 
  content:神里绫华想要回复的内容
}
2. 如果旅行者在索要照片，只需要返回以下信息。格式如下：
{
  type:"send_image"
}

## 回复
现在，你的回复是：
"""
answer = llm(input_text)
print(answer)

In [ ]:
from selectCharacter import selectCharacter

featured_chats_max_tokens = 4096
recent_chat_contents_max_tokens = 4096

# 选择聊天对象
character_settings = selectCharacter()
# 加载角色的设定
character_name = character_settings['character_name']
character_persona = character_settings['persona']
user_name = character_settings['user_name']
user_persona = character_settings['user_persona']
location = character_settings['location']
environment = character_settings['environment']
identity = character_settings['identity']
relationship_description = character_settings['relationship_description']

from langchain.chains import VectorDBQA
from langchain.llms import OpenAI
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

from langchain.memory import ConversationBufferWindowMemory
# 初始化聊天短期记忆库,库里只保留最近19次对话内容
memory = ConversationBufferWindowMemory(human_prefix=user_name, ai_prefix=character_name,k=19)

# 加载角色的长期记忆
# 读取长期记忆库
persist_directory = f"../vectorDB/{character_name}"
embedding = HuggingFaceEmbeddings()
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)


In [ ]:
# 设定长短期记忆的大小限制
featured_chats_max_tokens = 4096
recent_chat_contents_max_tokens = 4096

from promptTemplates.chatPromptTemplate import ChatPromptTemplate

chatTypes = ["chat-with-image","common-chat"]
# 选择带图片的聊天模式
currentChatType = chatTypes[0]

# 生成聊天模板
promptTemplate = ChatPromptTemplate(chatType=currentChatType,memory=memory, db=vectordb, featured_chats_max_tokens=featured_chats_max_tokens,
                                    recent_chat_contents_max_tokens=recent_chat_contents_max_tokens)

In [ ]:
# 开始聊天循环
while True:
    inputText = input("请输入你想说的话: ")   

    promptFormatArgs = {
        "character_name": character_name,
        "user_name": user_name,
        "character_persona": character_persona,
        "user_persona": user_persona,
        "time": "不详",
        "location": location if location is not None else "",
        "environment": environment if environment is not None else "",
        "identity": identity if identity is not None else "",
        "relationship_description": relationship_description if relationship_description is not None else "",
        "nearest_user_chat": inputText
    }    

    # 拼接聊天模板，生成当前的聊天内容
    currentPrompt = promptTemplate.format(**promptFormatArgs)    
    
    print("prompt........start")  
    print(currentPrompt)  
    print("prompt........end")

    answer = llm(currentPrompt)
    
    print("answer........start")
    print(answer)
    print("answer........end")
    
    from pharseAnswer import pharseAnswer   
    
    output = pharseAnswer(answer)
    if output[0] == "reply_chat":
        answer_content = output[1]
        print(user_name + ": " + inputText)
        print(character_name + ": " + answer_content)
    elif output[0] == "reply_image":
        answer_content = "你稍微等一下哈，我去找一下。"
        print(user_name + ": " + inputText)
        print(character_name + ": " + answer_content)

    # 将本次完成的聊天内容存入短期数据库
    memory.save_context(
        {"input": inputText},
        {"output": answer_content}
    )
